### Env Config

In [1]:
import os
while not os.path.exists('.gitignore'):
	os.chdir(os.path.abspath(os.path.join('../')))
from common_imports import *


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# Aproximação da quantidade de linhas esperadas no arquivo final 
TOTAL_ROW_COUNT = 100000

# Exportar os arquivos tratados localmente?
DO_EXPORT_FILES = True

In [3]:
def export_to_file(df: DataFrame, path: str):
	if not DO_EXPORT_FILES:
		return
	if os.path.exists(f'{path}/df.csv'):
		os.remove(f'{path}/df.csv')
	if os.path.exists(f'{path}/df.parquet'):
		try:
			os.remove(f'{path}/df.parquet')
		except:
			os.system(f'rm -rf {path}/df.parquet')
	df.to_csv(f'{path}/df.csv', index=False)
	df.to_parquet(f'{path}/df.parquet', index=False)

### Criando dataframe

In [4]:
files = os.listdir('_data/raw/operacoes-credito')
files.sort()

sample_size = TOTAL_ROW_COUNT // len(files)

arrow_tables = []
for file in files:
	print(f'Processando o arquivo {file}')

	temp_df = pd.read_parquet(f'_data/raw/operacoes-credito/{file}', engine='pyarrow').sample(n=sample_size, random_state=random.randint(1, 1000))
	table = pa.Table.from_pandas(temp_df)
	arrow_tables.append(table)

combined_table = pa.concat_tables(arrow_tables)

df = combined_table.to_pandas()

Processando o arquivo 201206.parquet
Processando o arquivo 201207.parquet
Processando o arquivo 201208.parquet
Processando o arquivo 201209.parquet
Processando o arquivo 201210.parquet
Processando o arquivo 201211.parquet
Processando o arquivo 201212.parquet
Processando o arquivo 201301.parquet
Processando o arquivo 201302.parquet
Processando o arquivo 201303.parquet
Processando o arquivo 201304.parquet
Processando o arquivo 201305.parquet
Processando o arquivo 201306.parquet
Processando o arquivo 201307.parquet
Processando o arquivo 201308.parquet
Processando o arquivo 201309.parquet
Processando o arquivo 201310.parquet
Processando o arquivo 201311.parquet
Processando o arquivo 201312.parquet
Processando o arquivo 201401.parquet
Processando o arquivo 201402.parquet
Processando o arquivo 201403.parquet
Processando o arquivo 201404.parquet
Processando o arquivo 201405.parquet
Processando o arquivo 201406.parquet
Processando o arquivo 201407.parquet
Processando o arquivo 201408.parquet
P

### Ajuste de tipagem dos dados

In [5]:
df.dtypes

data_base                         object
uf                                object
cliente                           object
ocupacao                          object
cnae_secao                        object
porte                             object
modalidade                        object
numero_de_operacoes               object
carteira_ativa                    object
carteira_inadimplida_arrastada    object
ativo_problematico                object
indexador_modalidade              object
dtype: object

In [6]:
df['cliente'].value_counts()

cliente
PJ    42954
PF     6978
Name: count, dtype: int64

In [7]:
df = df.rename(columns={
	'cliente': 'classificacao',
	'cnae_secao': 'cnae',
	'numero_de_operacoes': 'operacoes',
	'data_base': 'base',
})

In [8]:
print(df['porte'].value_counts())
print(df['modalidade'].value_counts())
print(df['ocupacao'].value_counts())
print(df['cnae'].value_counts())

porte
PJ - Pequeno                                     15730
PJ - Micro                                       15335
PJ - Médio                                        9780
PJ - Grande                                       1877
PF - Mais de 5 a 10 salários mínimos              1031
PF - Mais de 10 a 20 salários mínimos              930
PF - Mais de 3 a 5 salários mínimos                928
PF - Mais de 2 a 3 salários mínimos                872
PF - Acima de 20 salários mínimos                  869
PF - Mais de 1 a 2 salários mínimos                838
PF - Até 1 salário mínimo                          747
PF - Sem rendimento                                491
PF - Indisponível                                  272
PJ - Indisponível                                  232
Name: count, dtype: int64
modalidade
PJ - Capital de giro                                                              9974
PJ - Outros créditos                                                              8189
PJ - Cheque e

In [9]:
df['porte'] = df['porte'].str.slice(5).str.strip()
df['modalidade'] = df['modalidade'].str.slice(5).str.strip()
df['ocupacao'] = df['ocupacao'].str.slice(5).str.strip()
df['cnae'] = df['cnae'].str.slice(5).str.strip()

In [10]:
MONETARY_COLS = ['carteira_ativa', 'carteira_inadimplida_arrastada', 'ativo_problematico']

CATEGORY_COLS = ['uf', 'ocupacao', 'cnae', 'porte', 'modalidade', 'indexador_modalidade', 'classificacao']

QUANTITY_COLS = ['operacoes']

DATE_COLS = ['base']

for column in df.columns:
	if column in MONETARY_COLS:
		df[column] = df[column].str.replace(',', '.').astype(float)
		df = df.rename(columns={column: f'vl_{column}'})
	elif column in CATEGORY_COLS:
		df[column] = df[column].astype('category')
		df = df.rename(columns={column: f'ct_{column}'})
	elif column in QUANTITY_COLS:
		df[column] = df[column].str.replace('<= ', '').astype(int)
		df = df.rename(columns={column: f'qt_{column}'})
	elif column in DATE_COLS:
		df[column] = pd.to_datetime(df[column], format='%Y-%m-%d')
		df = df.rename(columns={column: f'dt_{column}'})

In [11]:
df.dtypes

dt_base                              datetime64[ns]
ct_uf                                      category
ct_classificacao                           category
ct_ocupacao                                category
ct_cnae                                    category
ct_porte                                   category
ct_modalidade                              category
qt_operacoes                                  int64
vl_carteira_ativa                           float64
vl_carteira_inadimplida_arrastada           float64
vl_ativo_problematico                       float64
ct_indexador_modalidade                    category
dtype: object

In [12]:
df = df[df['dt_base'] > (datetime.now() - timedelta(days=7*365))]

### Exportando para zona Trusted

In [13]:
export_to_file(df, '_data/trusted/operacoes-credito')

In [14]:
df['vl_carteira_problematica'] = df['vl_ativo_problematico'] - df['vl_carteira_inadimplida_arrastada']

In [15]:
df['vl_carteira_problematica'].describe()

count       27702.00
mean       218570.68
std       4766445.52
min      -4881383.23
25%             0.00
50%             0.00
75%          2715.41
max     453869956.06
Name: vl_carteira_problematica, dtype: float64

#### Valores menores do que 0?

In [16]:
df[df['vl_carteira_problematica'] < 0]['vl_carteira_problematica']

356096   -4881383.23
Name: vl_carteira_problematica, dtype: float64

#### 0.05% da amostra está danificada, vamos capar em 0

In [17]:
df['vl_carteira_problematica'] = df['vl_carteira_problematica'].clip(lower=0)

In [18]:
df['vl_carteira_problematica'].describe()

count       27702.00
mean       218746.89
std       4766347.20
min             0.00
25%             0.00
50%             0.00
75%          2715.41
max     453869956.06
Name: vl_carteira_problematica, dtype: float64

In [19]:
df['vl_carteira_ativa'] = df['vl_carteira_ativa'] - df['vl_carteira_problematica']
df.rename(columns={ 'vl_carteira_ativa': 'vl_carteira_saudavel' }, inplace=True)

In [20]:
df['vl_carteira_saudavel'].describe()

count         27702.00
mean        5381949.12
std       135512629.39
min               0.00
25%           15687.23
50%           89375.54
75%          490676.74
max     19854974695.23
Name: vl_carteira_saudavel, dtype: float64

In [21]:
df['vl_carteira_saudavel'] = df['vl_carteira_saudavel'] / df['qt_operacoes']
df['vl_carteira_problematica'] = df['vl_carteira_problematica'] / df['qt_operacoes']
df['vl_carteira_inadimplida_arrastada'] = df['vl_carteira_inadimplida_arrastada'] / df['qt_operacoes']

In [22]:
df.drop(columns=['qt_operacoes', 'vl_ativo_problematico'], inplace=True)

In [23]:
df['dt_ano'] = df['dt_base'].dt.year

In [24]:
df.dtypes

dt_base                              datetime64[ns]
ct_uf                                      category
ct_classificacao                           category
ct_ocupacao                                category
ct_cnae                                    category
ct_porte                                   category
ct_modalidade                              category
vl_carteira_saudavel                        float64
vl_carteira_inadimplida_arrastada           float64
ct_indexador_modalidade                    category
vl_carteira_problematica                    float64
dt_ano                                        int32
dtype: object

### Exportando para zona Refined

In [25]:
export_to_file(df, '_data/refined/operacoes-credito')